In [1]:
{-# LANGUAGE FlexibleContexts, MonadComprehensions, NoImplicitPrelude, RebindableSyntax, TypeFamilies #-}
import Language.Stochaskell
stochaskell

Stochaskell, version 0.1.0
Copyright (C) 2015-2019 David A Roberts
This program comes with ABSOLUTELY NO WARRANTY.
This is free software, and you are welcome to redistribute it
under certain conditions; see the LICENSE for details.

Using installation directory at 
  /home/jovyan/stochaskell

In [2]:
prior :: P (Z,RVec,RVec)
prior = do
  n <- poisson 5
  let base = uniform 0 5
  x <- orderedSample n base
  let mu = vector [ 0 | i <- 1...n ]
      kernel a b = exp (-(a - b)^2 / 2) + if a == b then 1e-6 else 0
      cov = matrix [ kernel (x!i) (x!j) | i <- 1...n, j <- 1...n ]
  y <- normal mu cov
  return (n,x,y)

In [3]:
let posterior = [ y | (n,x,y) <- prior, n == 5, x == list [0.5,1.5,2.5,3.5,4.5] ]

In [4]:
samples <- hmcStan 1000 posterior
print $ last samples

--- Generating Stan code ---
data {
  int x_stan_0_0;
  vector[x_stan_0_0] x_stan_0_1;
  
}
parameters {
  
  
  vector[x_stan_0_0] x_stan_0_2;
}
model {
  
  vector[x_stan_0_0] v_0_1;
  
  matrix[x_stan_0_0,x_stan_0_0] v_0_3;
  
  
  for (i_1_1 in 1:x_stan_0_0) {
  
    v_0_1[i_1_1] = 0;
  }
  
  for (i_1_1 in 1:x_stan_0_0) for (i_1_2 in 1:x_stan_0_0) {
    real v_1_0;
    real v_1_1;
    real v_1_2;
    real v_1_3;
    real v_1_4;
    int v_1_5;
    real v_1_6;
    real v_1_7;
    v_1_0 = x_stan_0_1[i_1_1] - x_stan_0_1[i_1_2];
    v_1_1 = v_1_0 .* v_1_0;
    v_1_2 = v_1_1 ./ 2;
    v_1_3 = -(v_1_2);
    v_1_4 = exp(v_1_3);
    v_1_5 = x_stan_0_1[i_1_1] == x_stan_0_1[i_1_2];
    v_1_6 = v_1_5 ? 1.0e-6 : 0;
    v_1_7 = v_1_4 + v_1_6;
    v_0_3[i_1_1, i_1_2] = v_1_7;
  }
  

  
  
  x_stan_0_2 ~ multi_normal(v_0_1, v_0_3);
}

make -C /home/jovyan/stochaskell/cmdstan /home/jovyan/stochaskell/cache/stan/model_674d3e4301d1e412a3dc977884d91470db35a6bc
make[1]: Entering directory '/home/jovy

[-1.5277,-1.83303,-0.756986,-0.317618,0.158873]

In [5]:
import Language.PyMC3
putStrLn $ pmProgram' defaultPyMC3Inference posterior Nothing
samples <- runPyMC3 defaultPyMC3Inference posterior Nothing
print $ last samples

import numpy as np
import pymc3 as pm
import sys
import time
import theano
import theano.tensor as tt
import theano.tensor.basic
import theano.tensor.slinalg
from theano.ifelse import ifelse

def ascolumn(a): return a.dimshuffle(0,'x')
def asrow(a):    return a.dimshuffle('x',0)
def quad_form_diag(m,v):
  d = theano.tensor.basic.diag(v)
  return d.dot(m).dot(d)
def lkj_corr(name, eta, n):
  C_triu = pm.LKJCorr(name + '_triu', eta=eta, n=n)
  shape = n * (n - 1) // 2
  tri_index = np.zeros([n, n], dtype='int32')
  tri_index[np.triu_indices(n, k=1)] = np.arange(shape)
  tri_index[np.triu_indices(n, k=1)[::-1]] = np.arange(shape)
  return pm.Deterministic(name, tt.fill_diagonal(C_triu[tri_index], 1))

theano.config.floatX = 'float32'

with pm.Model() as model:
  x_pm_0_0 = tt.as_tensor_variable(np.load('x_pm_0_0.npy').astype(theano.config.floatX))
  x_pm_0_0 = tt.as_tensor_variable(np.load('x_pm_0_0.npy').astype(theano.config.floatX))
  x_pm_0_1 = tt.as_tensor_variable(np.load('x_pm_0_1.n

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [x_pm_0_2]
100%|##########| 1000/1000 [00:02<00:00, 376.22it/s]

[0.19914907217025757,0.27238166332244873,0.3973581790924072,0.9212682247161865,1.5233206562697887e-2]

In [6]:
import Language.Edward
putStrLn $ edProgram 1000 10 0.3 posterior Nothing
samples <- hmcEdward 1000 10 0.3 posterior Nothing
print $ last samples

import sys
import time
from collections import OrderedDict
import edward as ed
import numpy as np
import tensorflow as tf

def all_equal(x,y): return tf.reduce_all(tf.equal(x,y))
def ascolumn(a): return tf.reshape(a, (-1, 1))
def asrow(a):    return tf.reshape(a, ( 1,-1))
def samples(inference, latent):
  s = []
  for x,q in latent.items():
    try:
      x_unconstrained = inference.transformations[x]
      params = x_unconstrained.bijector.inverse(q.params)
    except KeyError, AttributeError:
      sys.stderr.write(str(x) + ' is already unconstrained\n')
      params = q.params
    s.append(params.eval().tolist())
  return map(list, zip(*s))

with tf.Session().as_default():
  x_ed_0_0 = tf.constant(np.load('x_ed_0_0.npy'), dtype=tf.int32)
  x_ed_0_0 = tf.constant(np.load('x_ed_0_0.npy'), dtype=tf.float32)
  x_ed_0_1 = tf.constant(np.load('x_ed_0_1.npy'), dtype=tf.float32)
  v_0_3 = 0.0 * tf.ones([int(x_ed_0_0.eval())], dtype=tf.float32)
  def v_0_4_fn(i_1_1, i_1_2):
    v_1_0 = x_ed_

2019-09-05 16:35:53.145652: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2019-09-05 16:35:53.145715: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2019-09-05 16:35:53.145864: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
1000/1000 [100%] ██████████████████████████████ Elapsed: 8s | Acceptance Rate: 0.954

[2.4720120429992676,2.3773159980773926,0.5680612325668335,-3.847859799861908e-2,1.0919506549835205]